In [ ]:
import os, sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
%pip install --quiet "faiss-cpu>=1.7,<2" "ipywidgets>=7,<8" "pypdf>=3.8,<4"

1. Bedrock client 생성

In [ ]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region="us-east-1"
)

2. 챗봇(기본 - 컨텍스트 없음)

In [ ]:
from utils.chat import chat_utils
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
# - create the Anthropic Model
llm_text = Bedrock(
  model_id="anthropic.claude-v2",
  client=boto3_bedrock,
  model_kwargs={
    "max_tokens_to_sample": 512,
    "temperature": 0,
    "top_k": 250,
    "top_p": 0.999,
    "stop_sequences": ["\n\nHuman:"]
  },
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
memory= chat_utils.get_memory(
  memory_type="ConversationBufferMemory",
  memory_key='history'
)

conversation = ConversationChain(
  llm=llm_text,
  verbose=True,
  memory=memory
)

In [ ]:
if llm_text.streaming:
  response = conversation.predict(input="안녕하세요?")
else:
  print_ww(conversation.predict(input="안녕하세요?"))

### Reset Memory

In [ ]:
chat_utils.clear_memory(
  chain=conversation
)
print_ww(memory.load_memory_variables({}))

## 3. 프롬프트 템플릿(Langchain)을 이용한 챗봇

In [ ]:
from langchain import PromptTemplate

In [ ]:
# turn verbose to true to se the full logs and documents
conversation = ConversationChain(
  llm=llm_text,
  verbose=False,
  memory=memory
)

claude_prompt = PromptTemplate(
  input_variables = ["history", "input"],
  template="""
  \n\nHuman: Here's a friendly conversation between a user and an AI.
  The AI is talkative and provides lots of contextualized details.
  if it doesn't know. it will honestly say that it doesn't know the answer to the question.

  Current convesation:
  {history}

  \n\nUser: {input}

  \n\nAssistant:
  """
)
print("claude_prompt: \n", claude_prompt)

In [ ]:
conversation.prompt = claude_prompt

In [ ]:
chat_utils.get_tokens(
  chain=conversation,
  prompt="안녕하세요?"
)

In [ ]:
if llm_text.streaming:
  response = conversation.predict(input="안녕하세요")
else:
  print_ww(conversation.predict(input="안녕하세요?"))

In [ ]:
if llm_text.streaming:
  response = conversation.predict(input="좋은 고기를 고르는 몇 가지 팁을 알려 주세요")
else:
  print_ww(conversation.predict(input="좋은 고기를 고르는 몇 가지 팁을 알려 주세요"))

### 질문을 토대로 작성

모델이 이전 대화를 이해할 수 있는 지 확인하기 위해 정원이라는 단어를 언급하지 않고 질문.

In [ ]:
if llm_text.streaming:
  response = conversation.predict(input="좋아요. 생선에도 적용될까요?")
else:
  print_ww(conversation.predict(input="좋아요. 생선에도 적용될까요?"))

In [ ]:
print_ww(memory.load_memory_variables({}))

### (3) 대화 종료

In [ ]:
if llm_text.streaming:
  response = conversation.predict(input="좋은 정보 감사합니다. 고마워요!")
else:
  print_ww(conversation.predict(input="좋은 정보 감사합니다. 고마워요!"))

In [ ]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
  """A chat UX using IPYWidgets"""
  def __init__(self, qa, retrivealChain = False) -> None:
    self.qa = qa
    self.name = None
    self.b = None
    self.retrievalChain = retrivealChain
    self.out = ipw.Output()

    if "ConversationChain" in str(type(self.qa)):
      self.streaming = self.qa.llm.streaming
    elif "ConversationalRetrievalChain" in str(type(self.qa)):
      self.streaming = self.qa.combine_docs_chain.llm_chain.llm.streaming

  def start_chat(self):
    print("Starting chat bot")
    display(self.out)
    self.chat(None)

  def chat(self, _):
    if self.name is None:
      prompt = ""
    else:
      prompt = self.name.value
    if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
      print("Thank you, that was a nice chat !!")
      return
    elif len(prompt):
      with self.out:
        thiking = ipw.Label(value="Thinking...")
        display(thiking)
        try:
          if self.retrievalChain:
            result = self.qa.run({'question': prompt})
          else:
            result = self.qa.run({'input': prompt})
        except:
          result = "No answer"
        thiking.value = ""
        if self.streaming:
          response = f"AI:{result}"
        else:
          print_ww(f"AI:{result}")
        self.name.disabled = True
        self.b.disabled = True
        self.name=None
    
    if self.name is None:
      with self.out:
        self.name = ipw.Text(description="You:", placeholder='q to quit')
        self.b = ipw.Button(description="Send")
        self.b.on_click(self.chat)
        display(ipw.Box(children=(self.name, self.b)))

In [ ]:
chat =ChatUX(conversation)
chat.start_chat()

## 5. 페르소나를 활용한 챗봇
AI비서가 커리어 코치 역할을 하게 됩니다.
- 역할극 대화에서는 채팅을 시작하기 전에 사용자ㅏ 메시지를 설정해야 합니다. CoversationBufferMemory는 대화 상자를 미리 채우는데 사용됩니다

(1) ConversationChain 생성 필요한 메모리 초기화, Bedrock Claude 설정

In [ ]:
# llm

llm_text = Bedrock(
  model_id="anthropic.claude-v2",
  client=boto3_bedrock,
  model_kwargs={
    "max_tokens_to_sample": 1000,
    "temperature": 0,
    "top_k": 250,
    "top_p": 0.999,
    "stop_sequences": ["\n\nHuman:"]
  },
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

from langchain.memory import ConversationBufferMemory
# memory
# store previous interactions using ConversationalBufferMemory and add custom prompts to the that
memory = chat_utils.get_memory(
  memory_type="ConversationBufferMemory",
  memory_key="history"
)

memory.chat_memory.add_user_message("당신은 직업 코치로 활동하게 될 것입니다. 귀하의 목표는 사용자에게 직업 조언을 제공하는 것입니다.")
memory.chat_memory.add_ai_message("나는 직업 코치이며 직업에 대한 조언을 제공합니다.")

# conversation chain
conversation = ConversationChain(
  llm=llm_text,
  verbose=True,
  memory=memory
)

print(conversation)

# langchain prompts do not always work with the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""
\n\nHuman: Here's a friendly conversation between a user and AI.
The AI is talkative and provides lots of contextualized details.
If it doesn't know, it will honestly say that it doesn't know the answer to the question.

Current conversation:
{history}

User: {input}

\n\nAssistant:
""")

print("claude_prompt: \n", claude_prompt)
conversation.prompt = claude_prompt

In [ ]:
if llm_text.streaming:
  response = conversation.predict(input="인공지능에 관련된 직업은 어떤 것이 있습니까?")
else:
  print_ww(conversation.predict(input="인공지능에 관련된 직업은 어떤 것이 있습니까?"))

In [ ]:
if llm_text.streaming:
    response = conversation.predict(input="이 직업들은 실제로 무엇을 하는가요? 재미있나요?")
else:
    print_ww(conversation.predict(input="이 직업들은 실제로 무엇을 하는가요? 재미있나요?"))

In [ ]:
pprint(memory.load_memory_variables({}))

## 6. 맥락을 가진 챗봇

### 상황에 맞는 챗봇

이 사용 사례에서는 Chatbot에게 이전에 본 적이 없는 외부 코퍼스의 질문에 답변하도록 요청합니다. 이를 위해 RAG(Retrieval Augmented Generation)
이라는 패턴을 적용합니다. 아이디어는 말뭉치를 덩어리로 인덱싱한 다음 덩어리와 질문사이의 의미론적 유사성을 사용하여 말뭉치의 어느 섹션이 답변을 제공
하는 데 관련될 수 있는 지 찾는 것입니다. 마지막으로 가장 관련성이 높은 청크가 집계되어 기록을 제공하는 것과 유사하게 ConversationChain에 컨텍스트로
전달됩니다.

### 6.1 2022년

In [ ]:
from langchain.embeddings import BedrockEmbeddings

In [ ]:
bedrock_embeddings = BedrockEmbeddings(
  client=boto3_bedrock,
  model_id=bedrock_info.get_model_id(
    model_name="Titan-Embeddings-G1"
  )
)

PyPDFDirctoryLoader를 통한 PDF 파일 로딩
- chunk_size는 임베딩 모델의 최대 토큰인 512를 고려해서 정함.

In [ ]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

In [ ]:
# PDF 위치
loader = PyPDFDirectoryLoader("./rag_data_kr_pdf/")

In [ ]:
# loading documents and split 
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=230,
  chunk_overlap=50,
)
docs = text_splitter.split_documents(documents)

In [ ]:
# save to memory using FAISS
vectorstore_faiss_aws = FAISS.from_documents(
  documents = docs,
  embedding = bedrock_embeddings,
)

In [ ]:
print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents]) / len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

In [ ]:
print("docs[0].paget_content: \n", docs[0].page_content)

In [ ]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

### Semantic Search
LangChain에서 제공하는 Wrapper 클래스를 사용하여 벡터 데이터베이스 저장소를 쿼리하고 관련 문서를 변환.<br>
뒤에서는 RetrievalQA체인만 실행

In [ ]:
query = "아마존은 Generative AI의 전략은 무엇인가요"
print_ww(wrapper_store_faiss.query(query, llm=llm_text))

의미론적 검색

In [ ]:
v = bedrock_embeddings.embed_query(query)
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=3)
for r in results:
  print_ww(r.page_content)
  print('-----')

### 메모리
모든 챗봇에는 사용 사례에 따라 맞춤화된 다양한 옵션을 갖춘 QA 체인이 필요합니다. 그러나, 챗봇에서는 모델이 답변을 제공하기 위해 이를 고려할 수 있도록
항상 대화 기록을 보관해야 합니다. 이 예에서는 대화 기록을 유지하기 위해 ConversationBufferMemory와 함께 LangChain의 ConversationalRetrievalChain을
사용합니다.

In [ ]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
print_ww(CONDENSE_QUESTION_PROMPT.template)

#### ConversationalRetrievalChain에 사용되는 매개변수
- retriever: 우리는 VectorStore가 지원하는 VectorStoreRetriever를 사용. 텍스트를 검색하려면 "similarity" 또는 "mmr"라는 두 가지 검색 유형 선택
  search_type="similarity"는 질문 벡터와 가장 유사한 텍스트 청크 벡터를 선택하는 검색 객체에서 유사성 검색 사용
- 메모리: 이력을 저장하는 메모리 체인
- dense_question_prompt: 사용자의 질문이 주어지면 이전 대화와 해당 질문을 사용하여 독립형 질문을 구성
- chain_type: 채팅 기록이 길고 상황에 맞지 않는 경우, 이 매개 변수를 사용하고 stuff, refine, map_reduce, map_relank가 있다. 

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
  llm=llm_text,
  retriever=vectorstore_faiss_aws.as_retriever(),
  memory=memory_chain,
  condense_question_prompt=CONDENSE_QUESTION_PROMPT,
  chain_type='stuff'
)

In [ ]:
chat = ChatUX(qa, retrivealChain=True)
chat.start_chat()

기본 랭체인 프롬프트는 Claude에게 최적이 아닙니다. 다음 셀에서는 두 개의 새로운 프롬프트 설정
하나는 질문의 표현을 변경하기 위한 것이고, 다른 하나는 해당 질문의 답변을 얻기 위한 것.

In [ ]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

새로운 질문으로만 대답하세요.


Human: 이전 대화를 고려하여 질문을 어떻게 하시겠습니까?: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provide longer response but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. 
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: <q></q> XML 태그내의 질문에 답하려면 최대 3개의 문장을 사용하세요.

<q>{question}</q>

답변에 XML태그를 사용하지 마세요. 답변이 context에 없으면 "죄송합니다. 문맥에서 찾을 수 없어서 모르겠습니다."라고 말합니다.

Assistant:""")


채팅을 해보죠. 아래와 같은 질문을 해보세요.
1. 아마존의 Generative AI 의 전략이 무엇인가요?

In [ ]:
chat = ChatUX(qa, retrivealChain=True)
chat.start_chat()